In [0]:
# Configuring access to data lake
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "77a60835-4d15-4fbd-b8c2-f1635c31ea7b",
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="akv-secret-scope-kpmg",key="adls2-secret"),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d4e8a2a5-0301-4a81-9187-ecaaec70bd74/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://kpmgdata@sadatakpmg.dfs.core.windows.net/",
  mount_point = "/mnt/kpmg",
  extra_configs = configs)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-4493850945297489> in <module> 
 7 
 8 # Optionally, you can add <directory-name> to the source URI of your mount point. 
 ----> 9 dbutils.fs.mount(
 10 source = "abfss://kpmgdata@sadatakpmg.dfs.core.windows.net/" , 
 11 mount_point = "/mnt/kpmg" , 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 387 exc . __context__ = None 
 388 exc . __cause__ = None 
 --> 389 raise exc
 390 
 391 return f_with_exception_handling

 ExecutionError : An error occurred while calling o362.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/kpmg; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/kpmg
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:756)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:776)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/kpmg
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:465)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$1(MetadataManager.scala:830)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:611)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:819)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:473)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:93)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:103)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:102)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:102)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:318)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:277)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:120)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:147)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:147)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:102)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:413)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.

In [0]:
dbutils.fs.ls("/mnt/kpmg/")

Out[2]: [FileInfo(path='dbfs:/mnt/kpmg/decase3.xlsx', name='decase3.xlsx', size=4158684, modificationTime=1657102120000)]

In [0]:
import pandas as pd
from pyspark.sql.functions import *

file_path = 'dbfs:/mnt/kpmg/decase3.xlsx'
#xls = pd.ExcelFile(file_path)

customersDF = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("dataAddress", "'Customers'!A1") \
    .load(file_path)


customersDF = customersDF.select(col("customer_name"), col("customer_id"), col("gender"),to_date(col("customer_dob"),"yyyy-MM-dd").alias("date_of_birth"))


transactionsDF = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("dataAddress", "'Transactions'!A1") \
    .load(file_path)

transactionsDF = transactionsDF.select(col("transaction_id"), col("customer_id"), col("product_id"),col("store_id"),col("offer_id"),col("sales_amount"),to_date(col("transaction_date"),"yyyy-MM-dd").alias("date_transaction"))


offersDF = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("dataAddress", "'Offers'!A1") \
    .load(file_path)

offersDF = offersDF.select(col("customer_id"), col("offer_id"),to_date(col("start_date"),"yyyy-MM-dd").alias("date_start"), to_date(col("end_date"),"yyyy-MM-dd").alias("date_end"))

#df1 = pd.read_excel(xls, 'Sheet1')
#df2 = pd.read_excel(xls, 'Sheet2')

In [0]:
customersDF.show()
transactionsDF.show()
offersDF.show()
print(customersDF.count())
print(transactionsDF.count())
print(offersDF.count())

+-------------+-----------+------+-------------+
customer_name|customer_id|gender|date_of_birth|
+-------------+-----------+------+-------------+
 xxxx| 1187667.0| F| 1931-03-03|
 xxxx| 1188705.0| F| 1964-02-05|
 xxxx| 881223.0| M| 1944-10-18|
 xxxx| 882197.0| F| 1947-05-18|
 xxxx| 882210.0| M| 1970-06-05|
 xxxx| 882213.0| M| 1970-12-09|
 xxxx| 882224.0| F| 1964-02-18|
 xxxx| 882227.0| M| 1962-06-26|
 xxxx| 883208.0| M| 1936-05-31|
 xxxx| 883215.0| F| 1958-06-08|
 xxxx| 883259.0| F| 1945-07-08|
 xxxx| 828745.0| M| 1940-06-04|
 xxxx| 828774.0| F| 1944-06-11|
 xxxx| 834981.0| F| 1954-08-13|
 xxxx| 834993.0| F| 1947-02-18|
 xxxx| 835000.0| F| 1977-10-24|
 xxxx| 1320993.0| F| 1969-11-14|
 xxxx| 1270119.0| F| 1933-09-24|
 xxxx| 1189697.0| M| 1948-01-28|
 xxxx| 1189712.0| F| 1948-03-26|
+-------------+-----------+------+-------------+
only showing top 20 rows

+--------------+-----------+----------+--------+-------------+-------------------+----------------+
transaction_id|customer_id|product_id|store_id| offer_id| sales_amount|date_transaction|
+--------------+-----------+----------+--------+-------------+-------------------+----------------+
 1.1853308E7| 869573.0| 74069.0| 909.0|7.001370897E9| 18.247134428708367| 2020-03-14|
 5632404.0| 878676.0| 267295.0| 1262.0|2.000113555E9| 44.99822854898147| 2020-01-26|
 1.5234583E7| 891142.0| 82960.0| 134.0|7.001373027E9| 4.305763018322082| 2020-03-18|
 6325000.0| 1182772.0| 82715.0| 232.0| 466868.0| 104.42142779423318| 2020-03-31|
 5733971.0| 891142.0| 326249.0| 113.0|7.001367548E9| 0.6616945952324758| 2020-03-03|
 3.1902256E7| 1183811.0| 834549.0| 1190.0|7.001353021E9| 6.046884484913408| 2020-01-30|
 4.685257E7| 1148716.0| 447800.0| 1159.0|7.001376606E9|0.45395667708798565| 2020-04-07|
 1.015874E7| 1194597.0| 52330.0| 1413.0|7.001369991E9| 2.2828148289699977| 2020-03-10|
 1.1437378E7| 869606.0| 14051.0| 694.0|2.000124326E9| 567.2491424097929| 2020-03-25|
 4.3238293E7| 1188274.0| 69226.0| 1079.0| 463064.0| 139.55251784011787| 2020-02-15|
 7708582.0| 1493284.0| 86343.0| 661.0| 467221.0| 220.68347752568252| 2020-02-20|
 4942376.0| 868763.0| 83772.0| 108.0| 466454.0| 13.206420999856755| 2020-02-16|
 1.1890953E7| 869573.0| 71227.0| 909.0| null| 32.6667176772853| 2020-04-09|
 2.1725609E7| 1154788.0| 268085.0| 1260.0| null| 31.782601040714702| 2020-01-15|
 1.0836096E7| 1147729.0| 42977.0| 221.0| null| 55.26021592945064| 2020-04-08|
 3.0170708E7| 842701.0| 101509.0| 653.0| null| 5.735226433124156| 2020-02-25|
 7039508.0| 1278851.0| 693260.0| 1366.0| null|0.18446917462423912| 2020-01-28|
 7272787.0| 1154787.0| 887803.0| 869.0| null|0.13007243724114473| 2020-03-25|
 4014792.0| 1182104.0| 815825.0| 353.0| null| 1.9368686912021573| 2020-04-14|
 4621744.0| 865663.0| 800676.0| 1224.0| null| 5.746815066269003| 2020-03-10|
+--------------+-----------+----------+--------+-------------+-------------------+----------------+
only showing top 20 rows

None
+-----------+-------------+----------+----------+
customer_id| offer_id|date_start| date_end|
+-----------+-------------+----------+----------+
 1182119.0|2.000110064E9|2019-12-19|2019-12-29|
 1189159.0|2.000106346E9|2019-11-20|2019-12-08|
 1277982.0|7.001364884E9|2020-02-18|2020-02-19|
 887155.0|2.000141345E9|2020-04-22|2020-05-10|
 1192911.0|2.000126114E9|2020-03-18|2020-04-05|
 1315484.0|7.001355964E9|2020-01-08|2020-01-08|
 878657.0|7.001372786E9|2020-03-17|2020-03-17|
 1184093.0|7.001363638E9|2020-02-11|2020-02-11|
 875870.0|2.000110399E9|2019-12-18|2020-01-05|
 1184093.0|7.001361302E9|2020-01-28|2020-01-28|
 890059.0|7.001348432E9|2019-12-03|2019-12-03|
 841226.0|7.001350174E9|2019-12-10|2019-12-10|
 1298618.0|2.000113722E9|2020-01-20|2020-01-26|
 841226.0|2.000126057E9|2020-03-18|2020-04-05|
 1193238.0|7.001356031E9|2019-12-31|2019-12-31|
 871913.0| 2.00011432E9|2020-01-22|2020-02-09|
 1184858.0|7.001364839E9|2020-02-17|2020-02-23|
 878647.0| 7.00136705E9|2020-02-25|2020-02-25|
 852784.0|2.000106983E9|2019-11-20|2019-12-08|
 888105.0|2.000103061E9|20

In [0]:
# Define the input and output formats and paths and the table name.
write_format = 'delta'
save_path_customers = '/tmp/delta/customers_'
save_path_transactions = '/tmp/delta/transactions_'
save_path_offers = '/tmp/delta/offers_'
customers_table_name = 'default.customers'
transactions_table_name = 'default.transactions'
offers_table_name = 'default.offers'

# Write the data to its target.
customersDF.write \
  .format(write_format) \
  .save(save_path_customers)

transactionsDF.write \
  .format(write_format) \
  .save(save_path_transactions)

offersDF.write \
  .format(write_format) \
  .save(save_path_offers)

In [0]:
# Create the tables.
spark.sql("CREATE TABLE " + customers_table_name + " USING DELTA LOCATION '" + save_path_customers + "'")
spark.sql("CREATE TABLE " + transactions_table_name + " USING DELTA LOCATION '" + save_path_transactions + "'")
spark.sql("CREATE TABLE " + offers_table_name + " USING DELTA LOCATION '" + save_path_offers + "'")

Out[11]: DataFrame[]

In [0]:
%sql
SELECT * FROM default.customers LIMIT 20

customer_name,customer_id,gender,date_of_birth
xxxx,1187667.0,F,1931-03-03
xxxx,1188705.0,F,1964-02-05
xxxx,881223.0,M,1944-10-18
xxxx,882197.0,F,1947-05-18
xxxx,882210.0,M,1970-06-05
xxxx,882213.0,M,1970-12-09
xxxx,882224.0,F,1964-02-18
xxxx,882227.0,M,1962-06-26
xxxx,883208.0,M,1936-05-31
xxxx,883215.0,F,1958-06-08


In [0]:
%sql
SELECT * FROM default.transactions LIMIT 20

transaction_id,customer_id,product_id,store_id,offer_id,sales_amount,date_transaction
1.1853308E7,869573.0,74069.0,909.0,7.001370897E9,18.247134428708367,2020-03-14
5632404.0,878676.0,267295.0,1262.0,2.000113555E9,44.99822854898147,2020-01-26
1.5234583E7,891142.0,82960.0,134.0,7.001373027E9,4.305763018322082,2020-03-18
6325000.0,1182772.0,82715.0,232.0,466868.0,104.42142779423318,2020-03-31
5733971.0,891142.0,326249.0,113.0,7.001367548E9,0.6616945952324758,2020-03-03
3.1902256E7,1183811.0,834549.0,1190.0,7.001353021E9,6.046884484913408,2020-01-30
4.685257E7,1148716.0,447800.0,1159.0,7.001376606E9,0.45395667708798565,2020-04-07
1.015874E7,1194597.0,52330.0,1413.0,7.001369991E9,2.2828148289699977,2020-03-10
1.1437378E7,869606.0,14051.0,694.0,2.000124326E9,567.2491424097929,2020-03-25
4.3238293E7,1188274.0,69226.0,1079.0,463064.0,139.55251784011787,2020-02-15


In [0]:
%sql
SELECT * FROM default.offers LIMIT 20

customer_id,offer_id,date_start,date_end
1182119.0,2.000110064E9,2019-12-19,2019-12-29
1189159.0,2.000106346E9,2019-11-20,2019-12-08
1277982.0,7.001364884E9,2020-02-18,2020-02-19
887155.0,2.000141345E9,2020-04-22,2020-05-10
1192911.0,2.000126114E9,2020-03-18,2020-04-05
1315484.0,7.001355964E9,2020-01-08,2020-01-08
878657.0,7.001372786E9,2020-03-17,2020-03-17
1184093.0,7.001363638E9,2020-02-11,2020-02-11
875870.0,2.000110399E9,2019-12-18,2020-01-05
1184093.0,7.001361302E9,2020-01-28,2020-01-28


In [0]:
'''
1. Write a query that gives us an aggregated table that provides:
o Customer name,
o Customer id,
o Customer’s age (in years),
o Total sales per customer,  # numero de compras/transacoes de um cliente
o Number of offers received,  # numero de ofertas/promocoes recebidas pelo cliente
o Number of offers redeemed,  # numero de ofertas/promocoes que o cliente actually comprou
o Number of visits for each day of week (Mon-Sun), e.g. one column per weekday.  # quantas vezes o cliente foi num determinado dia da semana
'''

Out[13]: '\n1. Write a query that gives us an aggregated table that provides:\no Customer name,\no Customer id,\no Customer’s age (in years),\no Total sales per customer, # numero de compras/transacoes de um cliente\no Number of offers received, # numero de ofertas/promocoes recebidas pelo cliente\no Number of offers redeemed, # numero de ofertas/promocoes que o cliente actually comprou\no Number of visits for each day of week (Mon-Sun), e.g. one column per weekday. # quantas vezes o cliente foi num determinado dia da semana\n'

In [0]:
%sql
CREATE TABLE default.aggregated_table
AS 
(SELECT C.customer_name,
       C.customer_id,
       (SELECT FLOOR(DATEDIFF(current_date(), C.date_of_birth)/365)) AS customer_age,
       
       (SELECT COUNT(T.transaction_id) 
        FROM default.transactions T
        WHERE T.customer_id = C.customer_id
        GROUP BY T.customer_id) AS sales_per_customer,
        
        (SELECT COUNT(distinct(T.offer_id))
        FROM default.transactions T
        WHERE T.customer_id = C.customer_id
        GROUP BY T.customer_id) AS offers_received,
        
        (SELECT COUNT(T.transaction_id) 
        FROM default.transactions T
        WHERE T.customer_id = C.customer_id AND T.offer_id IS NOT null
        GROUP BY T.customer_id) AS offers_redeemed,
        
        (SELECT COUNT(T.transaction_id) 
        FROM default.transactions T
        WHERE T.customer_id = C.customer_id AND WEEKDAY(T.date_transaction) = 0
        GROUP BY T.customer_id) AS monday_visit,
        
        (SELECT COUNT(T.transaction_id) 
        FROM default.transactions T
        WHERE T.customer_id = C.customer_id AND WEEKDAY(T.date_transaction) = 1
        GROUP BY T.customer_id) AS tuesday_visit,
        
        (SELECT COUNT(T.transaction_id) 
        FROM default.transactions T
        WHERE T.customer_id = C.customer_id AND WEEKDAY(T.date_transaction) = 2
        GROUP BY T.customer_id) AS wednesday_visit,
        
        (SELECT COUNT(T.transaction_id) 
        FROM default.transactions T
        WHERE T.customer_id = C.customer_id AND WEEKDAY(T.date_transaction) = 3
        GROUP BY T.customer_id) AS thursday_visit,
        
        (SELECT COUNT(T.transaction_id) 
        FROM default.transactions T
        WHERE T.customer_id = C.customer_id AND WEEKDAY(T.date_transaction) = 4
        GROUP BY T.customer_id) AS friday_visit,
        
        (SELECT COUNT(T.transaction_id) 
        FROM default.transactions T
        WHERE T.customer_id = C.customer_id AND WEEKDAY(T.date_transaction) = 5
        GROUP BY T.customer_id) AS saturday_visit,
        
        (SELECT COUNT(T.transaction_id) 
        FROM default.transactions T
        WHERE T.customer_id = C.customer_id AND WEEKDAY(T.date_transaction) = 6
        GROUP BY T.customer_id) AS sunday_visit

FROM default.customers C);

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM default.aggregated_table;

customer_name,customer_id,customer_age,sales_per_customer,offers_received,offers_redeemed,monday_visit,tuesday_visit,wednesday_visit,thursday_visit,friday_visit,saturday_visit,sunday_visit
xxxx,883393.0,51,8,4,5,2,0,1,0,0,2,3
xxxx,883425.0,69,59,19,51,0,41,0,2,6,6,4
xxxx,883449.0,65,38,7,30,4,0,0,1,1,2,30
xxxx,1493093.0,80,0,0,0,0,0,0,0,0,0,0
xxxx,1494110.0,90,93,11,76,0,15,10,67,1,0,0
xxxx,1318820.0,73,0,0,0,0,0,0,0,0,0,0
xxxx,1318848.0,65,4,2,2,0,1,0,0,2,0,1
xxxx,1320012.0,78,69,23,53,0,0,68,0,0,1,0
xxxx,1320018.0,62,4,2,3,1,0,0,3,0,0,0
xxxx,1189884.0,72,56,24,47,1,28,2,16,9,0,0


In [0]:
'''
Additionally, write a query that tells us on which day of week that customers of the same age group (0-9 years, 10-19 years, etc.) visits the stores most often.
'''

In [0]:
%sql
SELECT t.range AS customer_range, sum(monday_visit) AS monday_visits, sum(tuesday_visit) AS tuesday_visits, sum(wednesday_visit) AS wednesday_visits, sum(thursday_visit) AS thursday_visits, sum(friday_visit) AS friday_visits, sum(saturday_visit) AS saturday_visits, sum(sunday_visit) AS sunday_visits
FROM (
  SELECT monday_visit, tuesday_visit, wednesday_visit, thursday_visit, friday_visit, saturday_visit, sunday_visit, 
  CASE 
    WHEN customer_age BETWEEN  0 AND  9 THEN '0-9'
    WHEN customer_age BETWEEN 10 AND 19 THEN '10-19'
    WHEN customer_age BETWEEN 20 AND 29 THEN '20-29'
    WHEN customer_age BETWEEN 30 AND 39 THEN '30-39'
    WHEN customer_age BETWEEN 40 AND 49 THEN '40-49'
    WHEN customer_age BETWEEN 50 AND 59 THEN '50-59'
    WHEN customer_age BETWEEN 60 AND 69 THEN '60-69'
    WHEN customer_age BETWEEN 70 AND 79 THEN '70-79'
    WHEN customer_age BETWEEN 80 AND 89 THEN '80-89'
    ELSE '90-100' END AS RANGE
  FROM default.aggregated_table) t
GROUP BY t.range

customer_range,monday_visits,tuesday_visits,wednesday_visits,thursday_visits,friday_visits,saturday_visits,sunday_visits
60-69,892,3883,2020,2638,1637,1051,725
80-89,418,3304,1649,1919,1168,502,86
40-49,374,426,308,577,469,400,169
90-100,66,788,488,334,454,505,69
70-79,1342,7871,3852,3210,2463,833,609
50-59,1198,1207,1478,1783,2360,1456,726


In [0]:
%sql
SELECT b.customer_range, GREATEST(b.monday_visits, b.tuesday_visits, b.wednesday_visits, b.thursday_visits, b.friday_visits, b.saturday_visits, b.sunday_visits) AS max_visits,
  CASE GREATEST(b.monday_visits, b.tuesday_visits, b.wednesday_visits, b.thursday_visits, b.friday_visits, b.saturday_visits, b.sunday_visits)
  WHEN b.monday_visits THEN 'monday'
  WHEN b.tuesday_visits THEN 'tuesday'
  WHEN b.wednesday_visits THEN 'wednesday'
  WHEN b.thursday_visits THEN 'thursday'
  WHEN b.friday_visits THEN 'friday'
  WHEN b.saturday_visits THEN 'saturday'
  ELSE 'sunday' END AS day
FROM (
SELECT t.range AS customer_range, sum(monday_visit) AS monday_visits, sum(tuesday_visit) AS tuesday_visits, sum(wednesday_visit) AS wednesday_visits, sum(thursday_visit) AS thursday_visits, sum(friday_visit) AS friday_visits, sum(saturday_visit) AS saturday_visits, sum(sunday_visit) AS sunday_visits
FROM (
  SELECT monday_visit, tuesday_visit, wednesday_visit, thursday_visit, friday_visit, saturday_visit, sunday_visit, 
  CASE 
    WHEN customer_age BETWEEN  0 AND  9 THEN '0-9'
    WHEN customer_age BETWEEN 10 AND 19 THEN '10-19'
    WHEN customer_age BETWEEN 20 AND 29 THEN '20-29'
    WHEN customer_age BETWEEN 30 AND 39 THEN '30-39'
    WHEN customer_age BETWEEN 40 AND 49 THEN '40-49'
    WHEN customer_age BETWEEN 50 AND 59 THEN '50-59'
    WHEN customer_age BETWEEN 60 AND 69 THEN '60-69'
    WHEN customer_age BETWEEN 70 AND 79 THEN '70-79'
    WHEN customer_age BETWEEN 80 AND 89 THEN '80-89'
    ELSE '90-100' END AS RANGE
  FROM default.aggregated_table) t
GROUP BY t.range) AS b ORDER BY b.customer_range;

customer_range,max_visits,day
40-49,577,thursday
50-59,2360,friday
60-69,3883,tuesday
70-79,7871,tuesday
80-89,3304,tuesday
90-100,788,tuesday
